In [1]:
from glob import glob 

for g in glob('../data/*.pdf'):
    print(g)

../data\2040_seoul_plan.pdf
../data\OneNYC_2050_Strategic_Plan.pdf


In [2]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

def read_pdf_and_split_text(pdf_path, chunk_size=1000, chunk_overlap=100):
    """
    주어진 PDF 파일을 읽고 텍스트를 분할합니다.
    매개변수:
        pdf_path (str): PDF 파일의 경로.
        chunk_size (int, 선택적): 각 텍스트 청크의 크기. 기본값은 1000입니다.
        chunk_overlap (int, 선택적): 청크 간의 중첩 크기. 기본값은 100입니다.
    반환값:
        list: 분할된 텍스트 청크의 리스트.
    """
    print(f"PDF: {pdf_path} -----------------------------")

    pdf_loader = PyPDFLoader(pdf_path)
    data_from_pdf = pdf_loader.load()

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap
    )

    splits = text_splitter.split_documents(data_from_pdf)
    
    print(f"Number of splits: {len(splits)}\n")
    return splits


In [3]:
%pip install langchain_huggingface

Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

Looking in indexes: https://download.pytorch.org/whl/cu124
Note: you may need to restart the kernel to use updated packages.


In [5]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3",
    model_kwargs = {'device': 'cuda'}, # cuda 지원하면 cuda, 아닌 경우 cpu
    encode_kwargs = {'normalize_embeddings': True}, # 임베딩 정규화
)

c:\github\gpt_agent_2025_easyspub\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
embeddings.embed_documents("안녕하세요")

[[0.0024569211527705193,
  0.03226257115602493,
  -0.007424186449497938,
  0.005268432199954987,
  -0.05809735134243965,
  -0.03042859397828579,
  -0.0083000548183918,
  0.03674239292740822,
  0.009622692130506039,
  -0.007634009700268507,
  0.017204446718096733,
  0.03887706995010376,
  -0.029649008065462112,
  -0.021820083260536194,
  -0.0008699358440935612,
  -0.032562464475631714,
  0.03178009018301964,
  -0.02246295101940632,
  0.02350258082151413,
  -0.012333727441728115,
  -0.038810212165117264,
  -0.017907671630382538,
  0.038729291409254074,
  0.013715717941522598,
  0.025562603026628494,
  0.02299593761563301,
  -0.02759174071252346,
  0.02707342803478241,
  -0.00977579690515995,
  -0.026509804651141167,
  -0.006837872322648764,
  -0.0293024480342865,
  0.028823232278227806,
  -0.07535611093044281,
  -0.032743774354457855,
  -0.00434119114652276,
  -0.023116158321499825,
  0.021545181050896645,
  -0.05429798364639282,
  0.0528126060962677,
  0.043331366032361984,
  -0.0208815

In [7]:
from langchain_chroma import Chroma

import os
persist_directory='../chroma_store'

if os.path.exists(persist_directory):
    print("Loading existing Chroma store")
    vectorstore = Chroma(
        persist_directory=persist_directory, 
        embedding_function=embeddings
    )
else:
    print("Creating new Chroma store")
    
    all_splits = []
    for g in glob('../data/*.pdf'):
        all_splits.extend(read_pdf_and_split_text(g))

    print(f"Total number of splits: {len(all_splits)}")

    vectorstore = Chroma.from_documents(
        documents=all_splits,
        embedding=embeddings,
        persist_directory=persist_directory
    )

Creating new Chroma store
PDF: ../data\2040_seoul_plan.pdf -----------------------------
Number of splits: 308

PDF: ../data\OneNYC_2050_Strategic_Plan.pdf -----------------------------
Number of splits: 1023

Total number of splits: 1331


c:\github\gpt_agent_2025_easyspub\venv\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\saint\.cache\huggingface\hub\models--BAAI--bge-m3. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [8]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

chunks = retriever.invoke("서울시 쓰레기 저감 정책")

for chunk in chunks:
    print(chunk.metadata)
    print(chunk.page_content)

{'author': 'SI', 'creationdate': '2024-12-12T18:16:11+09:00', 'creator': 'Hwp 2020 11.0.0.5178', 'moddate': '2024-12-12T18:16:11+09:00', 'page': 64, 'page_label': '65', 'pdfversion': '1.4', 'producer': 'Hancom PDF 1.3.0.542', 'source': '../data\\2040_seoul_plan.pdf', 'total_pages': 205}
제3절 2040 서울도시기본계획 7대 목표57Ÿ특히, 서울시는 현재 온실가스 배출량의 90%를 차지하고 있는 건물과 수송 부문 감축을 위해 적극적인 대책을 마련하고 있다.-2026년까지 건물 에너지효율화사업 100만 호를 추진, 건물온실가스총량제, 신규건물 제로에너지 건물(ZEB) 의무화를 통해 기존 및 신규건물의 제로에너지화를 촉진-수송 부문 배출감축을 위해 전기차 비중을 2026년까지 10%(’21년 5.2만 대 → ’26년 40만 대)로 확대하고, 22만기의 충전 인프라 구축을 계획 중(’21년 2만기) 시민 생활과 안전을 위한 기후위기 대응 요구 심화Ÿ서울은 인구, 시설 등이 밀집해 있는 대도시로 기온 상승, 폭염, 집중호우, 태풍, 한파 등 극한 기후 현상이 더욱 빈번하게 발생할 것으로 전망된다. 이러한 기후위험은 서울시민의 일상생활과 안전을 크게 위협할 수 있다. 따라서 시민의 일상을 보호하고 도시의 회복력을 강화하기 위한 적극적인 기후위기 대응 전략이 필요하다.Ÿ특히 다양화·복합화되는 재난안전사고에 대응해 전통적인 자연·사회재난의 범주뿐 아니라, 신종 복합재난까지 대비할 수 있는 다면적인 대응체계 마련이 요구된다.2) 추진전략탄소중립·기후위기 적응대책은 시의 모든 정책과 사업의 주요 원칙으로 고려Ÿ탄소중립은 돌이키기 어려운 기후재난을 막기 위한 국제사회와의 약속이기에 시의 모든 정책과 사업의 결정 과정에서 주요하게 고려해야 할 포괄적 ‘원칙’